# Factor Evaluation: Portfolio Construction

I use the first four packages for generic python stuff. The math package is useful for the rand function. I have not used yfinance here but have left it in, as it will be very useful for visualisations later on. I've also used the pandas_datareader, though there are alternatives. The warning message is supressed due to the way I had to save the excel file for input - though I have checked to ensure that everything is entered correctly. Finally, for plotting I use plotly, which allows for interactive graphs. Particularly they have good support for financial visualisations.


In [114]:
#!pip install yfinance
#!pip install pandas_datareader
#!pip install tabulate 

import numpy as np 
import scipy as sp
import pandas as pd
import math
import random
import tabulate 

#external data & file reading
import yfinance as yf
import pandas_datareader as pdr
pd.options.mode.chained_assignment = None  # default='warn'


#for plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

I use the mappings file for what I believe to be a full list of the companies. This may be narrowed down in the excel file which could be easier, but this will be simple to change if needed. I took these excel files from the dropbox "Factors". 

In [3]:
mappings = pd.read_excel('mappings.xlsx', index_col=0)  

# Portfolio Selection Process

What I'm trying to do is document in the pdf. It is split into a few little sets of code, and I expect each of which will require changing! Here I define parameters for my testing.

In [4]:
n = 200 #the total number of stocks to invest in

target_sectors = ["Real_Estate","Industrials","Materials"] #these are the target sectors. Discussed later on. 
t_n_primed = 5 #this is the total number of sectors to include

#selects the regions to remove. Initially I test assuming we only want to look at stocks with market classification "Asia". 
regions = ["Europe","America"]


## Section 1 - Selecting the Regions

I've assumed that individuals may be interested in investing in regions they are interested in. This would be nice, as it would allow us to further narrow down candidate stocks (which will make the selection process easier later on). I leave in an option if an individual does not care how many regions they invest in. I may introduce classifications by the world bank later on. This would allow the selection of developing vs developed, or by gross national income for further customisation. 

In [5]:
#drops na's - this occured for 7 entries
all_regions = mappings['market'].unique()
mappings = mappings.dropna()

#removes the required regions. This is in-efficient, and i bet there is a one line simple function to do it. 
for i in range (0,len(regions)):
    mappings = mappings.drop(mappings[regions[i] == mappings['market']].index)

df_1 = mappings

## Section 2 - How many sectors? (with favouritism towards a few)

I assume people will likely have bias, and so I wanted to allow them to select specific sectors for which their portfolios would be weighted towards. I allow for up to 3 specified "target-sectors". I also allow for preference as to the number of total sectors let this be $t_n$. Denote the number of target-sectors as $t_s$. I initially chose that if someone specified target sectors, this would represent 80% of the portfolio. Then mathematically for now I have used sector weightings 

$$ \text{Weightings : } \begin{cases}\frac{1}{t_s} \cdot 0.8 \text{ for target sectors} \\  \frac{1}{t_n - t_s}\cdot 0.2 \text{ for non-target sectors} \end{cases} $$

Where $\sum \frac{1}{t_s}, \sum \frac{1}{t_n-t_s} = 1$ if we sum over the target sectors. This ensures weightings sum to one.


One thing I wonder is how we could go about selecting the non-target sectors. I do this randomly for now, though note that this would be an interesting thing to look at! It would be easy to do as we could generally just produce a list of candidate sectors for a point in time, and then run through each one to see if we'd already included it 

In [1]:
#number of total sectors

#this sets stuff up. t_n is the total number of industries, t_n_primed represents the maximum number we wish to include. 
all_sectors = df_1['Layer 1'].unique()
t_n =  len(all_sectors)
weightings = np.zeros(13)
t_s = len(target_sectors)

#this bit selects the non-target sectors, and should be more efficient. 
potential_sectors = []
for i in range(0,len(all_sectors)):
    if all_sectors[i] not in target_sectors:
        potential_sectors.append(all_sectors[i])
remaining_sectors = random.sample(potential_sectors,t_n_primed-t_s)
        

#this gets our weightings as described above. 
for i in range(0,t_n):
    s_1 = all_sectors[i]
    if (s_1 in target_sectors):
        weightings[i] = 1/t_s*0.8
    elif (s_1 in remaining_sectors):
        #this should be more advanced. In here we can better select the sectors
        weightings[i] = 1/(len(remaining_sectors))*0.2
    else:
        weightings[i] = 0 

final_sectors = target_sectors + remaining_sectors
df_2 = pd.DataFrame({'Sector':all_sectors,'Weighting':weightings,'# of stocks': np.floor(weightings*200)})

df_2

#note - there are minor rounding errors. It will have 199 instead of 200. Can easily be changed (I think...).

NameError: name 'df_1' is not defined

## Section 3 - Narrowing down candidate stocks

At this point we have narrowed down our sector and markets, but are still left with approx 20,000 stocks. This of course depends on the test data. Now within these subsets, we must further narrow down to determien the actual stocks we should invest in. At this point we introduce the factor analysis, and $\textbf{initially, I only use factor analysis on stock selection not for the sectors. This can easily be changed} $. This is split into a few subsections

- Firstly, setting our data up. This creates a list of dataframes, which we will then use to select our stocks from. Each dataframe represents a set of data for a region and sector pair. This could go further to country and (region and sector pair).

- Secondly, for each dataframe we determine which stocks to select. This is first based on factor analysis, and from there by comparing (other metrics here, volatility, VAR, downdraws etc).


At this point, I am cross comparing between excel files. This is in-efficient and at some point I'd like to try and find out if there is a file containing all the information in the same place. 

In [7]:
#Data Set-up
df_3 = df_1 

df_list = []
for i in final_sectors:
    df_list.append(df_3[df_3['Layer 1'] == i])
df_list = pd.concat(df_list)
df_list = df_list.reset_index()

countries_with_trade_info = ['CHINA','HONG KONG','JAPAN','KOREA','TAIWAN']
trades_list = []
for i in countries_with_trade_info:
    trades_list.append(pd.read_csv(i + ".csv"))
all_trades  = pd.concat(trades_list)

#Importing the  required excel files. This file contains the factor analysis for individual stocks. This takes a while to run!
#I renamed the trade files. For example "China Trades" to "CHINA" for simplicity. 

#Number of companies in asia we have trade data on
len(all_trades['fsym_id'].unique())
#total companies in asia matching our criteria.
len(df_list)

#Total companies which match our chosen sectors, and we have trade data on
len(list(set(df_list['fsym_id']) & set(all_trades['fsym_id'])))

df_f = df_list[df_list.fsym_id.isin(list(set(df_list['fsym_id']) & set(all_trades['fsym_id'])))]
df_trades = all_trades[all_trades.fsym_id.isin(df_f['fsym_id'].unique())]

In [32]:
#Now begins classication and categorisation by factors. Initially I just look at volatility, growth and liquidity / size.

factors = ["Volatility","Liq and Size",'Growth',"Momentum",'Dividends']

df_factors = []

#creates dataframes for each factor, stores in list
for i in range(0,len(factors)):
    df_factors.append(df_trades[df_trades.Factor == factors[i]])
    
#There are 2011 suitable stocks, and approx 1700 match each criteria. Further classification is needed? 
df_f2 = pd.concat(df_factors)
#len(df_f2['fsym_id'].unique()) 

2011

At this point, we have the dataframes of stock names for suitable countries for our pairs. We also have all the trades associated. We now need to consider risk profiles, and actually determining which of these stocks we wish to select. 

## Section 4 - Identify Candidate Stocks

## Section 5 - Stock weightings

## Section 6 - Historic returns, plots & metrics of our chosen portfolio

## Section 7 - Visualisations

I wanted to make some nice Spyder diagrams, which show the historic returns. This next cell makes the excel file easier to read.


In [46]:
xls = pd.ExcelFile('Stock Returns Annual.xlsx')
by_factor = pd.read_excel(xls, 'Factor Returns')
by_country = pd.read_excel(xls, 'Country Returns')
by_sector = pd.read_excel(xls, 'Sector-Factor Returns')
by_sec_and_count = pd.read_excel(xls, 'Country-Sector-Factor Returns')


for i in range (0,len(by_sec_and_count)):
    if isinstance(by_sec_and_count['country'].iloc[i], float):
        if math.isnan(by_sec_and_count['country'].iloc[i]) == True:
            by_sec_and_count['country'].iloc[i] = by_sec_and_count['country'].iloc[i-1]
            
    if isinstance(by_sec_and_count['Sector'].iloc[i], float):      
        if math.isnan(by_sec_and_count['Sector'].iloc[i]) == True:
            by_sec_and_count['Sector'].iloc[i] = by_sec_and_count['Sector'].iloc[i-1]   

### Spyder Diagrams 

This first one varies the year. in the loop add in which years you want plotted. This visualisation is not good, there are little trends and it is not that interesting.

In [102]:
#I now re-scale for plotting purposes. I want to work with only 2021 data initially
sector = 'Utilities'
country = 'AUSTRALIA'
year = 2021

country = by_sec_and_count[by_sec_and_count.country == country]
country_sector = country[country.Sector == sector]
country_sector = country_sector.set_axis(['Country','Sector','Factor','2017','2018','2019','2020','2021'], axis=1, inplace=False)

fig = go.Figure(data=go.Scatterpolar(
  r = country_sector['2021'],
  theta=country_sector['Factor'],
  fill='toself'
))

for i in [2020,2021]:
    fig.add_trace(go.Scatterpolar(
      r = country_sector[str(i)],
      theta=country_sector['Factor'],
      fill='toself',
      name='Product B'
))
    
fig.update_layout(title='Spyder Diagram: Returns for Australian Utilities, 2020 & 2021' ,
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()

country_sector

,Country,Sector,Factor,2017,2018,2019,2020,2021
72,AUSTRALIA,Utilities,Behavioural,0.000159,-0.000023,-0.000757,-0.002791,-0.000089
73,AUSTRALIA,Utilities,Dividends,0.000000,0.000000,0.000000,0.000000,0.000000
74,AUSTRALIA,Utilities,Growth,0.000000,0.000000,0.000000,0.000000,0.000000
75,AUSTRALIA,Utilities,Liq and Size,-0.000122,-0.000495,0.000915,-0.000407,0.000950
76,AUSTRALIA,Utilities,Momentum,-0.000194,0.000413,0.000395,0.000719,0.000322
77,AUSTRALIA,Utilities,Quality,0.000000,0.000000,0.000000,0.000000,0.000000
78,AUSTRALIA,Utilities,Value,-0.000351,-0.000025,0.000857,0.000414,-0.001063
79,AUSTRALIA,Utilities,Volatility,0.000092,0.000002,0.000297,0.000157,-0.000607


### Spyder 2

This one now looks at the same year, same sector, same factor, but varying region. This is a bit more interesting.

In [107]:
sector = "Utilities"
year = 2021
factor = "Volatility"

country_sector = by_sec_and_count[by_sec_and_count.Sector == sector]
country_sector = country_sector[country_sector.factors == factor]
country_sector = country_sector.set_axis(['Country','Sector','Factor','2017','2018','2019','2020','2021'], axis=1, inplace=False)



fig = go.Figure(data=go.Scatterpolar(
  r = country_sector['2019'],
  theta=country_sector['Country'],
  fill='toself'
))

for i in [2018,2020]:
    fig.add_trace(go.Scatterpolar(
      r = country_sector[str(i)],
      theta=country_sector['Country'],
      fill='toself',
      name='Product B'
))
    
    
fig.update_layout(title='Spyder Diagram: Volatility Returns: 2018,19 & 20' ,
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()


,Country,Sector,Factor,2017,2018,2019,2020,2021
79,AUSTRALIA,Utilities,Volatility,0.000092,0.000002,0.000297,0.000157,-0.000607
183,CHINA,Utilities,Volatility,0.000879,0.001317,0.000484,0.000592,0.001325
407,GERMANY,Utilities,Volatility,-0.000080,0.000933,0.000013,0.000171,0.000195
527,ITALY,Utilities,Volatility,0.001039,-0.000718,0.001745,-0.000669,0.000205
607,JAPAN,Utilities,Volatility,-0.000163,0.004004,0.001154,0.001041,-0.000003
743,KOREA,Utilities,Volatility,0.000000,0.001251,-0.000182,0.001611,0.000000
1023,UNITED KINGDOM,Utilities,Volatility,0.000778,0.000461,0.000496,0.000472,-0.000004
1103,UNITED STATES,Utilities,Volatility,-0.000046,0.001162,0.000423,0.000325,-0.000597


In [122]:
ticker_tag = 'TSM'

ticker = yf.Ticker(ticker_tag)

recommendations = ticker.recommendations
recommendations = recommendations.reset_index()

earnings = ticker.earnings
earnings = earnings.reset_index()

df = ticker.history(period="max")

df = df.reset_index()
#Converting the datatype to float
for i in ['Open', 'High', 'Close', 'Low']:
    df[i] = df[i].astype('float64')
    
    fig = go.Figure(data=go.Ohlc(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close']))



recomend = recommendations[recommendations.Firm == "Morgan Stanley"]
recomend['Change'] = recomend['From Grade'] + " to " + recomend['To Grade']
recomend = recomend.reset_index(drop=True)
shapes = []
annotations = [] 

for i in range (0,len(recomend)):
    date = recomend['Date'].iloc[i]
    date = pd.to_datetime(date.date())
    action =recomend['Action'].iloc[i]
    data_action_day = df[df.Date == date]

    shapes.append(dict(x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper',line_width=2))
    annotations.append(dict(x=date, y=0.05, xref='x', yref='paper',showarrow=False, xanchor = "left",text=action))
    

fig.update_layout(title='Historic Prices & Morgan Stanley (credit agency?) Evaluation Actions',
        yaxis_title=ticker_tag + ' Stock',
        shapes = shapes,
    annotations=annotations)




fig.show()

print(recomend.to_markdown())


#I want to make this plot nicer, i.e. make these bars more attractive!!

|    | Date                | Firm           | To Grade     | From Grade   | Action   | Change                      |
|---:|:--------------------|:---------------|:-------------|:-------------|:---------|:----------------------------|
|  0 | 2013-04-14 12:20:30 | Morgan Stanley | Equal-Weight | Overweight   | down     | Overweight to Equal-Weight  |
|  1 | 2016-02-18 08:05:19 | Morgan Stanley | Overweight   | Equal-Weight | up       | Equal-Weight to Overweight  |
|  2 | 2017-11-27 16:49:52 | Morgan Stanley | Equal-Weight | Overweight   | down     | Overweight to Equal-Weight  |
|  3 | 2018-11-16 12:13:29 | Morgan Stanley | Equal-Weight | Underweight  | up       | Underweight to Equal-Weight |
|  4 | 2019-09-05 10:28:24 | Morgan Stanley | Overweight   | Equal-Weight | up       | Equal-Weight to Overweight  |


,Date,Firm,To Grade,From Grade,Action,Change
3,2013-04-14 12:20:30,Morgan Stanley,Equal-Weight,Overweight,down,Overweight to Equal-Weight
9,2016-02-18 08:05:19,Morgan Stanley,Overweight,Equal-Weight,up,Equal-Weight to Overweight
14,2017-11-27 16:49:52,Morgan Stanley,Equal-Weight,Overweight,down,Overweight to Equal-Weight
18,2018-11-16 12:13:29,Morgan Stanley,Equal-Weight,Underweight,up,Underweight to Equal-Weight
21,2019-09-05 10:28:24,Morgan Stanley,Overweight,Equal-Weight,up,Equal-Weight to Overweight
